<a href="https://colab.research.google.com/github/kolomichyk/BKIT/blob/develop/Po_sqrt_team_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
from seaborn import pairplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from statistics import mean
from sklearn.metrics import mean_absolute_error

###Предобработка данных

In [21]:
data_target = pd.read_csv('/content/train_target.csv')

In [22]:
data = pd.read_parquet('/content/train.parquet')

In [23]:
def addAnswer(data):
  for j in range(0, data.shape[0], 19):
    NotNullElement  = False
    posNotNullElement = 0
    for i in range(j+18, j-1, -1):
      if (data['milleage_all'][i] and not NotNullElement):
        NotNullElement = True
        posNotNullElement = i
      if (data['milleage_all'][i]):
        data['answer'][i] = data_target['target'][j//19] +  data['milleage_all'][posNotNullElement] - data['milleage_all'][i]

Количество пропусков в данных

In [24]:
data['answer'] = [0 for i in range(data.shape[0])]

In [25]:
addAnswer(data)

<ipython-input-23-a71db577cb51>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['answer'][i] = data_target['target'][j//19] +  data['milleage_all'][posNotNullElement] - data['milleage_all'][i]


Удаляем строки с пропусками

In [12]:
df = data
df = df.dropna(axis=0)
df = df.drop(columns=['wagnum', 'ts_id'])
df.shape

(620886, 10)

In [13]:
df.isnull().sum()

milleage_all       0
axl1_l_w_flange    0
axl1_r_w_flange    0
axl2_l_w_flange    0
axl2_r_w_flange    0
axl3_l_w_flange    0
axl3_r_w_flange    0
axl4_l_w_flange    0
axl4_r_w_flange    0
answer             0
dtype: int64

Нормализуем данные

In [26]:
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

### Визуализируем данные

In [ ]:
vdata1 = df_normalized[:1000]

In [ ]:
pairplot(vdata1)

In [ ]:
vdata1 = vdata1.assign(avg1=vdata1[['axl1_l_w_flange', 'axl1_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg2=vdata1[['axl2_l_w_flange', 'axl2_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg3=vdata1[['axl3_l_w_flange', 'axl3_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg4=vdata1[['axl4_l_w_flange', 'axl4_r_w_flange']].mean(axis=1))

In [ ]:
pairplot(vdata1.drop(columns=['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange',
       'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange',
       'axl4_l_w_flange', 'axl4_r_w_flange']))

### Train test split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df_normalized.drop(columns=['answer']), df['answer'], test_size=0.20, random_state=42)

### Построение MVP модели

In [ ]:
linear_model = LinearRegression().fit(X_train, y_train)

In [ ]:
linear_model.score(X_test, y_test)

0.30000308810642407

In [ ]:
linear_model.coef_

array([2503.8054303 , 3616.34705899, 3419.74625692, 4271.59964096,
       3283.19374453, 3178.51222309, 4354.90039529, 2946.96618875,
       3692.85747515])

In [ ]:
lassoModel = Lasso(alpha = i * 0.25, tol = 0.0925)
for i in range(1, 9):
    lassoModel.fit(X_train, y_train)
    scores = cross_val_score(lassoModel, X_train, y_train, cv = 10)
    avg_cross_val_score = mean(scores)*100

In [ ]:
lassoModel.score(X_test, y_test)

0.3000033264738413

In [ ]:
lassoModel.coef_

array([2500.46019163, 3616.86815621, 3418.48137326, 4268.77719003,
       3280.05339539, 3176.46754717, 4354.25616113, 2947.88565021,
       3694.55949233])

In [ ]:
avg_cross_val_score

30.131320657597914

In [ ]:
mean_absolute_error(y_test, lassoModel.predict(X_test))

23848.71555115389

In [ ]:
mean_absolute_error(y_test, linear_model.predict(X_test))

23848.742280832463

### RandomForestRegressor

In [ ]:
Forest_regressor = RandomForestRegressor(100, criterion="absolute_error", max_depth=15)

In [ ]:
for epoch in range(1):
  Forest_regressor.fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_test, Forest_regressor.predict(X_test))

### XGBoost

In [28]:
import xgboost as xgb

In [29]:
xgb_r = xgb.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 42)

In [18]:
xgb_r.fit(X_train, y_train)

[05:50:14] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [19]:
mean_absolute_error(y_test, xgb_r.predict(X_test))

20146.426607667923

### Вторая теория (Усредним значения)

In [ ]:
def make_data(data):
  for j in range(0, data.shape[0], 19):
    NotNullElement  = False
    posNotNullElement = 0
    for i in range(j+18, j-1, -1):
      if (data['milleage_all'][i] and not NotNullElement):
        NotNullElement = True
        posNotNullElement = i
      if (data['milleage_all'][i]):
        data['answer'][i] = data_target['target'][j//19] +  data['milleage_all'][posNotNullElement] - data['milleage_all'][i]